In [1]:
def sum(*args):
    tot = 0
    for arg in args:
        tot+=arg
    return tot

In [2]:
print(sum(23,43,234.32,32.34,123/12))

342.90999999999997


In [3]:

def add(x: int, y: int) -> int:

    """return sum of ints only"""
    return x+y

from typing import get_type_hints

def validate_return(func, value):
    hints = get_type_hints(func)   # always returns a dict

    if 'return' in hints:
        return isinstance(value, hints['return'])

    return True


In [4]:
print(add.__annotations__)

print(validate_return(add, 20))   # True
print(validate_return(add, "20")) # False

{'x': <class 'int'>, 'y': <class 'int'>, 'return': <class 'int'>}
True
False


In [5]:
# default args:
notg = object()
def fun(a, b=notg):
    if b is notg:
        print('no b')
        return ""
    else:
        print(f"a is {a}, b is {b}")

print(fun(21))
print(fun(32,32))

no b

a is 32, b is 32
None


In [6]:
#7.7. Capturing Variables in Anonymous Functions
class Solution:
    def create_functions(self, n: int):
        return [lambda x, i=i : x + i for i in range(n)]

In [7]:
funcs = Solution().create_functions(7)
print(funcs)
results = [f(10) for f in funcs]
print(f"results for x=10: {results}")

[<function Solution.create_functions.<locals>.<lambda> at 0x10cf6b5e0>, <function Solution.create_functions.<locals>.<lambda> at 0x10cf6b690>, <function Solution.create_functions.<locals>.<lambda> at 0x10cf6b740>, <function Solution.create_functions.<locals>.<lambda> at 0x10cf6b7f0>, <function Solution.create_functions.<locals>.<lambda> at 0x10cf6b8a0>, <function Solution.create_functions.<locals>.<lambda> at 0x10cf6b950>, <function Solution.create_functions.<locals>.<lambda> at 0x10cf6ba00>]
results for x=10: [10, 11, 12, 13, 14, 15, 16]


In [8]:
# distnace based sorting of points:

import math
from functools import partial 

def distance(p1,p2):
    x1,y1 = p1
    x2,y2 = p2
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

print(distance((0,0), (5,5)))

class Solution:
    def sort_by_distance(self, points, origin):
        dist_from_origin = partial(distance, origin)
        return sorted(points, key = dist_from_origin)

sol = Solution()
pts = [(3,4), (1,2),(10,10), (0,1), ]
print(f"soted points = {sol.sort_by_distance(pts, origin=(0,0))}")

7.0710678118654755
soted points = [(0, 1), (1, 2), (3, 4), (10, 10)]


In [9]:
class Solution2:
    def url_temp_gen(self, template: str):
        def opener(**kwargs):
            return template.format_map(kwargs)

        return opener

sol = Solution2()
url_fmt = 'http://finance.yahoo.com/d/quotes.csv?s={names}&f={fields}'

yahoo = sol.url_temp_gen(url_fmt)

result = yahoo(names = "IBM, AAPL, FB", fields = "sl1c1v")

print(f"Generated URL: {result}")


Generated URL: http://finance.yahoo.com/d/quotes.csv?s=IBM, AAPL, FB&f=sl1c1v


In [15]:
def add(x, y):
    return x + y

def apply_async(func, args, *, callback):
    result = func(*args)
    callback(result)

class ResultHandler:
    def __init__(self):
        self.sequence = 0

    def handler(self, result):
        self.sequence += 1
        print(f"{self.sequence} got {result}")


def make_handler():
    sequence = 0
    def handler(result):
        nonlocal sequence
        sequence+=1
        print('[{}] Got: {}'.format(sequence, result))
    return handler
r = ResultHandler()
apply_async(add, (2, 3), callback=r.handler)
apply_async(add, (10, 5), callback=r.handler)

handler = make_handler()
apply_async(add, (2, 3), callback=handler)
apply_async(add, (10, 5), callback=handler)

1 got 5
2 got 15
[1] Got: 5
[2] Got: 15


In [28]:
import types

def add(x, y):
    return x + y

def apply_async(func, args, *, callback):
    result = func(*args)
    callback(result)  

def inline_async(func):
    def wrapper(*args):
        gen = func(*args)
        result = None

        try:
            future = gen.send(None)
            future(lambda res: wrapper_step(gen, res))
        except StopIteration:
            pass

    def wrapper_step(gen, res):
        try:
            future = gen.send(res)
            future(lambda next_res: wrapper_step(gen, next_res))
        except StopIteration:
            pass

    return wrapper

def make_handler(func, args):
    return lambda callback: apply_async(func, args, callback=callback)

@inline_async
def test():
    r1 = yield make_handler(add, (2, 3))
    r2 = yield make_handler(add, (r1, 10))
    print(f"got r2: {r2}")

test()

got r2: 15


In [30]:
# 7.12. Accessing Variables Defined Inside a Closure
# Problem
# You would like to extend a closure with functions that allow the inner variables to be
# accessed and modified.

def make_counter():
    count = 0

    def counter():
        nonlocal count
        count +=1
        return count

    def get_count():
        return count
    
    def set_count(value):
        nonlocal count
        count = value 

    counter.get_count = get_count
    counter.set_count = set_count 

    return counter 

c = make_counter() 
print(f" Nomrally call 1: {c()}")
print(f" Normal call 2: {c()}")

c.set_count(0)
print(c.get_count())

 Nomrally call 1: 1
 Normal call 2: 2
0
